In [2]:
import torch.nn as nn
import torch
from PIL import Image
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os

In [3]:
# Step 1
# Get the image and change every image to an 256-dimention vector
dataSet = np.zeros([10, 256])
for i in range(0, 10):
    inputImageDir = './input/' + str(i) + '.png'
    inputImage = Image.open(inputImageDir)
    inputImage = inputImage.convert("1")
    inputImage.save(inputImageDir)
    data = inputImage.getdata()
    array = np.array(data)/255
    dataSet[i] = array
dataSet = np.array(dataSet)

In [4]:
# Step 2
# define a neural network
class Perceptron(nn.Module):

    def __init__(self, input_size, num_hidden, num_classes):
        super(Perceptron, self).__init__()
        self.inputlayer = nn.Linear(input_size, num_hidden)
        self.hiddenlayer = nn.Linear(num_hidden, num_hidden)
        self.outputlayer = nn.Linear(num_hidden, num_classes)
        self.activate = nn.Sigmoid()

    def forward(self, x):
        res = self.inputlayer(x)
        res = self.activate(res)
        res = self.hiddenlayer(x)
        res = self.activate(res)
        res = self.outputlayer(x)
        res = self.activate(res)
        return res

In [5]:
class DigitDataset(Dataset):
    def __init__(self, dataset, label_list):
        self.dataset = dataset
        self.label_list = label_list
    def __len__(self):
        return len(self.label_list)
    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.label_list[idx]
        return {
            'data': torch.from_numpy(data).float(),
            'label': torch.from_numpy(label).float()
        }

In [6]:
# Parameters of training
input_size = 256
num_classes = 256
num_hidden = 256
learning_rate = 0.001
batch_size = 10
num_epochs = 600

# Load the dataset
train_dataset = DigitDataset(dataset = dataSet, label_list = dataSet)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
device = torch.device('cpu')
model = Perceptron(input_size=input_size, num_hidden=num_hidden, num_classes=num_classes).to(device)

if not os.path.exists('./models'):
    os.mkdir('./models')
torch.save(model, './models/net_untrained.pkl')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [7]:
# Step 3
def train(dataloader, model, num_epochs):
    for epoch in range(num_epochs):
        losses = []
        ERROR_Train = [] 
        model.train() 
        for i, data in enumerate(dataloader, 0):
            model.zero_grad()
            real_cpu, label_cpu = data['data'], data['label']
#             if torch.cuda.is_available():
#                 real_cpu = real_cpu.cuda() 
#                 label_cpu = label_cpu.cuda()
            real = real_cpu
            label = label_cpu
            inputv = Variable(real)
            labelv = Variable(label)
            output = model(inputv)
            err = criterion(output, labelv) 
            err.backward() 
            optimizer.step() 

            losses.append(err.data.item())
            error = label - output.data
#             print(error.shape)
            ERROR_Train.extend(error)
#         print(ERROR_Train)
        MAE = torch.mean(torch.abs(torch.stack(ERROR_Train)))
        ME = torch.mean(torch.stack(ERROR_Train))
        STD = torch.std(torch.stack(ERROR_Train)) 
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print('[%d/%d] Loss: %.4f MAE: %.4f Mean Error: %.4f STD: %.4f' % (epoch, num_epochs, np.average(losses), MAE, ME, STD))
    return output, model

# Start training        
output, model = train(train_loader, model, num_epochs)
# print(output.type)

[0/600] Loss: 0.2617 MAE: 0.4941 Mean Error: 0.4047 STD: 0.3130
[10/600] Loss: 0.0435 MAE: 0.1581 Mean Error: 0.0970 STD: 0.1847
[20/600] Loss: 0.0303 MAE: 0.0915 Mean Error: 0.0328 STD: 0.1711
[30/600] Loss: 0.0279 MAE: 0.0775 Mean Error: 0.0204 STD: 0.1657
[40/600] Loss: 0.0261 MAE: 0.0726 Mean Error: 0.0165 STD: 0.1608
[50/600] Loss: 0.0246 MAE: 0.0702 Mean Error: 0.0153 STD: 0.1563
[60/600] Loss: 0.0233 MAE: 0.0682 Mean Error: 0.0139 STD: 0.1519
[70/600] Loss: 0.0220 MAE: 0.0667 Mean Error: 0.0132 STD: 0.1477
[80/600] Loss: 0.0208 MAE: 0.0654 Mean Error: 0.0128 STD: 0.1436
[90/600] Loss: 0.0196 MAE: 0.0638 Mean Error: 0.0123 STD: 0.1396
[100/600] Loss: 0.0186 MAE: 0.0622 Mean Error: 0.0121 STD: 0.1358
[110/600] Loss: 0.0176 MAE: 0.0605 Mean Error: 0.0118 STD: 0.1320
[120/600] Loss: 0.0166 MAE: 0.0590 Mean Error: 0.0114 STD: 0.1284
[130/600] Loss: 0.0157 MAE: 0.0575 Mean Error: 0.0112 STD: 0.1249
[140/600] Loss: 0.0149 MAE: 0.0560 Mean Error: 0.0109 STD: 0.1215
[150/600] Loss: 0.014

In [8]:
# Step 4
# Step 4a
# Export the image after training
# Before executing this block, create a folder called "output"
if not os.path.exists('./output'):
    os.mkdir('./output')
output_np = output.detach().numpy()
print(output_np.shape)
torch.save(model, './models/net_trained.pkl')
output_dataset = np.zeros([10, 256])
for i in range(10):
    output_img = output_np[i].reshape(16, 16)*255
    img = Image.fromarray(np.uint8(output_img))
    img = img.convert("1")
    output_path = './output/' + str(i) + '.png'
    img.save(output_path)
    data = img.getdata()
    array = np.array(data)/255
    output_dataset[i] = array

(10, 256)


In [9]:
# Step 4b
# Calculate Fh
def calculateFh(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Fh_denominator = 0    # Fh分母
    Fh_numerator = 0      # Fh分子
    Fh_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 0:
                Fh_denominator = Fh_denominator + 1
                if output_dataset[j][i] == 0:
                    Fh_numerator = Fh_numerator + 1
        Fh = Fh_numerator / Fh_denominator
        Fh_array[j] = Fh
    return Fh_array

In [10]:
# Calculate Ffa
def calculateFfa(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Ffa_denominator = 0    # Ffa分母
    Ffa_numerator = 0      # Ffa分子
    Ffa_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 1:
                Ffa_denominator = Ffa_denominator + 1
            if output_dataset[j][i] == 0 and input_dataset[j][i] == 1:
                Ffa_numerator = Ffa_numerator + 1
        Ffa = Ffa_numerator / Ffa_denominator
        Ffa_array[j] = Ffa
    return Ffa_array

In [11]:
Fh_array = calculateFh(dataSet, output_dataset)
Ffa_array = calculateFfa(dataSet, output_dataset)
print(Fh_array)
print(Ffa_array)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.         0.         0.         0.0010627  0.00085106 0.00071073
 0.0006105  0.0005322  0.00047438 0.00042753]


In [12]:
# TODO: Step 4c: Graph Fh as a function of Ffa for each exemplar in the input dataset

In [13]:
# Step 5
def gaussian_noise(img, mean, sigma):
    # Generate gauss noise
    noise = np.random.normal(mean, sigma, img.shape)
    # Add the noise to image
    gaussian_out = img + noise
    # Make the value between 0 and 1
    gaussian_out = np.clip(gaussian_out, 0, 1)
    return gaussian_out

gaussian_dataset = np.zeros([9, 10, 256])
std = [0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
if not os.path.exists('./input_noise/'):
    os.mkdir('./input_noise/')
for j in range(9):
    if not os.path.exists('./input_noise/' + str(std[j])):
        os.mkdir('./input_noise/' + str(std[j]))
    for i in range(10):
        inputImage = dataSet[i]
        gaussian_data = gaussian_noise(inputImage, 0, std[j])
        img = gaussian_data.reshape(16, 16)*255
        img = Image.fromarray(np.uint8(img))
        img.convert("1")
        inputImageDir = './input_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(inputImageDir)
        gaussian_dataset[j][i] = gaussian_data
gaussian_dataset = np.array(gaussian_dataset)


Fh_noise_array = np.zeros([9, 10])
Ffa_noise_array = np.zeros([9, 10])

# Train 9 datasets with noise
if not os.path.exists('./output_noise/'):
    os.mkdir('./output_noise/')
for j in range(9):
    train_noise_dataset = DigitDataset(dataset = gaussian_dataset[j], label_list = dataSet)
    train_noise_loader = DataLoader(dataset=train_noise_dataset, batch_size=batch_size, shuffle=False)
    print('Training dataset with noise standard deviation ' + str(std[j]))
#     model_noise = torch.load('./models/net_untrained.pkl') #  Load the model that trained before
    model_noise = Perceptron(input_size=input_size, num_hidden=num_hidden, num_classes=num_classes).to(device)
    output_noise, model_noise = train(train_noise_loader, model_noise, num_epochs)   # Train
    torch.save(model_noise, './models/net_trained_' + str(std[j]) + '.pkl')
    model_noise = torch.load('./models/net_trained_' + str(std[j]) + '.pkl')
#     output_noise = model_noise(torch.from_numpy(gaussian_dataset[j]).float()) # Use the model trained before to test
    print('------------------------------------')
    output_noise = model_noise(torch.from_numpy(dataSet).float())
    output_noise_np = output_noise.detach().numpy()     # Get the output
#     print(output_noise_np)
    output_noise_dataset = np.zeros([10, 256])
#     Make the output only has 0 or 1
    
    if not os.path.exists('./output_noise/' + str(std[j])):
        os.mkdir('./output_noise/' + str(std[j]))
    
    
    
    for i in range(10):
        output_noise_img = output_noise_np[i].reshape(16, 16)*255
        img = Image.fromarray(np.uint8(output_noise_img))
        img = img.convert("1")
        output_path = './output_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(output_path)
        data = img.getdata()
        array = np.array(data)/255
        output_noise_dataset[i] = array
#     Calculate Fh and Ffa
    Fh = calculateFh(dataSet, output_noise_dataset)
    Ffa = calculateFfa(dataSet, output_noise_dataset)
    Fh_noise_array[j] = Fh
    Ffa_noise_array[j] = Ffa
print('------------Fh_noise_array------------')
print(Fh_noise_array)
print('------------Ffa_noise_array------------')
print(Ffa_noise_array)

Training dataset with noise standard deviation 0.001
[0/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[10/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[20/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[30/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[40/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[50/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[60/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[70/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[80/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[90/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[100/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[110/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[120/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[130/600] Loss: 0.2678 MAE: 0.5015 Mean Error: 0.4119 STD: 0.3133
[140/600] Loss: 0.2678 MAE: 0.5015

[0/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[10/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[20/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[30/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[40/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[50/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[60/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[70/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[80/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[90/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[100/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[110/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[120/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[130/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[140/600] Loss: 0.2640 MAE: 0.4990 Mean Error: 0.4130 STD: 0.3057
[150/600] Loss: 0.264

[40/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[50/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[60/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[70/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[80/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[90/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[100/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[110/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[120/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[130/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[140/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[150/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[160/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[170/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[180/600] Loss: 0.2655 MAE: 0.4984 Mean Error: 0.4145 STD: 0.3061
[190/600] Loss: 

[60/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[70/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[80/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[90/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[100/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[110/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[120/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[130/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[140/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[150/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[160/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[170/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[180/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[190/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[200/600] Loss: 0.2599 MAE: 0.4933 Mean Error: 0.4083 STD: 0.3054
[210/600] Loss

[70/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[80/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[90/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[100/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[110/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[120/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[130/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[140/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[150/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[160/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[170/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[180/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[190/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[200/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[210/600] Loss: 0.2624 MAE: 0.4955 Mean Error: 0.4146 STD: 0.3009
[220/600] Los

In [13]:
# TODO: Step 6: Display Data from your Tests in Step 5